<a href="https://colab.research.google.com/github/tarasryb/algohouse_sdk/blob/master/samples/algohouse_sdk_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/tarasryb/algohouse_sdk.git

In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
# !tar xvzf ta-lib-0.4.0-src.tar.gz
# import os
# os.chdir('ta-lib') # Can't use !cd in co-lab
# !./configure --prefix=/usr
# !make
# !make install
# os.chdir('../')
# !pip install TA-Lib


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import talib as ta
import pandas as pd
import requests

import sys
sys.path.append("algohouse_sdk")
import algohouse as ah
import ah_settings as ahs

# Settings

In [ ]:
USER_EMAIL = 'tarasryb@gmail.com'
SIGNKEY = '9566c74d10037c4d7bbb0407d1e2c649'

exchange = 'binance/f'
instrument = 'APEBUSD'  #'1000LUNCBUSD'
start_time = '2022-11-01T00:00:00'
end_time = '2022-12-30T23:00:00'

plotly_theme = "plotly_dark"  # "plotly_white"


# Reference Data

In [ ]:
ref_data = ah.get_reference_data()
ref_data.head(100)

In [ ]:
bnc_ref = ref_data.loc[ref_data['exchange'] == 'binance/f']
(bnc_ref['instruments'].values)

# Trades Without Aggregation

In [ ]:
trades = ah.get_trades(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=start_time,
                                        to_time=end_time)
trades = trades.loc[trades['bs'] == 'S']
trades.tail(100)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=trades['ts'], y=trades['price'], name="price"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=trades['ts'], y=trades['volume'], name="volume"),
    secondary_y=True,
)

fig.update_layout(
    title = f"{instrument} Price & Volume",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>volume</b>", secondary_y=True)

fig.show()

# Aggregated Trades

In [ ]:
trades_a = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=start_time,
                                        to_time=end_time,
                                        aggregation='1d')
trades_a.head(1000)

In [ ]:
fig = go.Figure(data=[
    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price")
    ])
fig.update_layout(
    title = f"{instrument} Price",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False
)

fig.show()


# Technical Analysis

## Volume Indicators

### Get Data

In [ ]:
window_start = '2022-11-20T00:00:00'
window_end = '2022-12-01T00:00:00'
df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=window_start,
                                        to_time=window_end,
                                        aggregation='1d')


In [ ]:
ver_window_start = '2022-12-01T00:00:00'
ver_window_end = '2022-12-12T00:00:00'
ver_df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=ver_window_start,
                                        to_time=ver_window_end,
                                        aggregation='1d')


In [ ]:
open = df['open']
high = df['high']
low = df['low']
close = df['close']
volume = df['volume']


### [AD - Chaikin A/D Line](https://www.investopedia.com/articles/trading/08/accumulation-distribution-line.asp)


In [ ]:
ad = ta.AD(high, low, close, volume)
print(ad)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['ts'], y=df['volume'], name="volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=ver_df['ts'], y=ver_df['volume'], name="verification volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=ver_df['ts'], y=ad, name="AD volume"),
    secondary_y=False,
)

fig.update_layout(
    title = f"{instrument} Chaikin A/D Line",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>volume</b>", secondary_y=False)
# fig.update_yaxes(title_text="<b>volume</b>", secondary_y=True)

fig.show()

### [ADOSC - Chaikin A/D Oscillator](https://www.investopedia.com/articles/active-trading/031914/understanding-chaikin-oscillator.asp)

In [ ]:
adosc = ta.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
print(adosc)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['ts'], y=df['volume'], name="volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=ver_df['ts'], y=ver_df['volume'], name="verification volume"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=adosc, name="Chaikin A/D Oscillator"),
    secondary_y=True,
)

fig.update_layout(
    title = f"{instrument} Chaikin A/D Oscillator",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>Volume</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Chaikin A/D Oscillator</b>", secondary_y=True)

fig.show()

###  [OBV - On Balance Volume](https://www.investopedia.com/terms/o/onbalancevolume.asp)

In [ ]:
ovb = ta.OBV(close, volume)
# print(ovb)

In [ ]:
ver_price_df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=window_start,
                                        to_time=ver_window_end,
                                        aggregation='1d')


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['ts'], y=df['volume'], name="volume"),
    secondary_y=False,
)

# fig.add_trace(
#     go.Scatter(x=ver_df['ts'], y=ver_df['volume'], name="verification volume"),
#     secondary_y=False,
# )

fig.add_trace(
    go.Scatter(x=ver_price_df['ts'], y=ver_price_df['close'], name="verification price"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=ovb, name="OVB volume"),
    secondary_y=False,
)

fig.update_layout(
    title = f"{instrument} On Balance Volume",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>volume</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>price</b>", secondary_y=True)

fig.show()

### Basic Indicators
* [ADV - Average Daily Value Traded*](https://www.investopedia.com/terms/a/averagedailytradingvolume.asp)
* MA - Moving Average

In [ ]:
idc = pd.DataFrame()
idc['ts'] = df['ts'] 
idc['volume'] = df['volume'] 
idc['ADV5d'] = df['volume'].rolling(window=5).mean().shift(1)
idc['ADV21d'] = df['volume'].rolling(window=21).mean().shift(1)
idc['MA1w'] = ta.MA(close, timeperiod=7, matype=0)
idc['MA3w'] = ta.MA(close, timeperiod=7*3, matype=0)
idc['VWAP'] = pd.Series(ta.WMA(close, timeperiod=24)) * df['volume']
# idc.head()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    go.Candlestick(x=df['ts'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['volume'], name="volume", mode = 'lines'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['ADV5d'], name="ADV 5d", mode = 'lines'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['ADV21d'], name="ADV 21d", mode = 'lines'),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['MA1w'], name="MA 1w", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['MA3w'], name="MA 3w", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=idc['ts'], y=idc['VWAP'], name="VWAP", mode = 'lines+markers'),
    secondary_y=True,
)

fig.update_layout(
    title = f"{instrument} Basic Indicators",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>ATR</b>", secondary_y=True)

fig.show()

## Volatility Indicators

In [ ]:
window_start = '2022-11-01T00:00:00'
window_end = '2022-12-12T00:00:00'
df = ah.get_trades_aggregated(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=window_start,
                                        to_time=window_end,
                                        aggregation='1d')
open = df['open']
high = df['high']
low = df['low']
close = df['close']
volume = df['volume']


### [ATR - Average True Range](https://www.investopedia.com/terms/a/atr.asp)

In [ ]:
atr = ta.ATR(high, low, close, timeperiod=14)
# print(atr)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=atr, name="ATR", mode = 'lines+markers'),
    secondary_y=True,
)


fig.update_layout(
    title = f"{instrument} Average True Range Indicator",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>ATR</b>", secondary_y=True)

fig.show()

### [NATR - Normalized Average True Range](https://support.atas.net/en/knowledge-bases/2/articles/43436-atr-normalized)

In [ ]:
natr = ta.NATR(high, low, close, timeperiod=14)
# print(natr)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=natr, name="NATR", mode = 'lines+markers'),
    secondary_y=True,
)


fig.update_layout(
    title = f"{instrument} Normalized Average True Range Indicator",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>NATR</b>", secondary_y=True)

fig.show()

### [TRANGE - True Range](https://support.atas.net/en/knowledge-bases/2/articles/45183-true-range)

In [ ]:
trange = ta.TRANGE(high, low, close)
# print(trange)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=trange, name="TRANGE", mode = 'lines+markers'),
    secondary_y=True,
)


fig.update_layout(
    title = f"{instrument} True Range Indicator",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>TRANGE</b>", secondary_y=True)

fig.show()

## Price Transform Functions
* AVGPRICE - Average Price
* MEDPRICE - Median Price
* TYPPRICE - Typical Price
* WCLPRICE - Weighted Close Price


In [ ]:
avgprice = ta.AVGPRICE(open, high, low, close)
medprice = ta.MEDPRICE(high, low)
typprice = ta.TYPPRICE(high, low, close)
wclprice = ta.WCLPRICE(high, low, close)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    
    # go.Scatter(x=df['ts'], y=df['close'], name="volume"),

    go.Candlestick(x=trades_a['ts'],
                open=trades_a['open'],
                high=trades_a['high'],
                low=trades_a['low'],
                close=trades_a['close'], name="OCHL price"),

    secondary_y=False,


)

fig.add_trace(
    go.Scatter(x=df['ts'], y=avgprice, name="AVGPRICE", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=medprice, name="MEDPRICE", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=typprice, name="TYPPRICE", mode = 'lines+markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['ts'], y=wclprice, name="WCLPRICE", mode = 'lines+markers'),
    secondary_y=False,
)


fig.update_layout(
    title = f"{instrument} Price Transform Functions",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>time</b>")
fig.update_yaxes(title_text="<b>price</b>", secondary_y=False)
# fig.update_yaxes(title_text="<b>TRANGE</b>", secondary_y=True)

fig.show()

# Order book

#### Reset orderbook cache

In [ ]:
import orderbook
orderbook.df_n_cached = None

#### Settings

In [ ]:
USER_EMAIL = 'tarasryb@gmail.com'
SIGNKEY = '9566c74d10037c4d7bbb0407d1e2c649'

exchange = 'binance/f'
instrument = 'LITUSDT'  #'BTCUSDT'#'ETHUSDT'
start_time = '2022-12-09T10:00:00'
ahs.ORDER_LINES_TO_READ = 200
# ahs.FILTER_BAD_PRICES = False

plotly_theme = "plotly_dark"  # "plotly_white"


#### Get Orderbook Data

In [ ]:
md, md_c = ah.get_orderbook(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=start_time,
                                        levels=None)

In [ ]:
bid = md_c['bid']
ask = md_c['ask']

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=bid['price'], y=bid['size'], name="bid", fill='tozeroy'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=ask['price'], y=ask['size'], name="ask", fill='tozeroy'),
    secondary_y=False,
)



fig.update_layout(
    title = f"{exchange} : {instrument} Market Depth at {start_time}",
    autosize=False,
    width=1200,
    height=600,
    template=plotly_theme,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=35,
        pad=4
    ),
    xaxis_rangeslider_visible=False,
)
fig.update_xaxes(title_text="<b>Price Level</b>")
fig.update_yaxes(title_text="<b>Size</b>", secondary_y=False)

fig.show()

In [ ]:
ask

In [ ]:
res = ah.get_orderbook(USER_EMAIL, SIGNKEY,
                                        instrument=instrument,
                                        exchange=exchange,
                                        from_time=start_time,
                                        levels=None, snapshot=True)
s = res['snapshot'] 

In [ ]:
s

# Streaming

In [ ]:
USER_EMAIL = 'tarasryb@gmail.com'
SIGNKEY = '9566c74d10037c4d7bbb0407d1e2c649'

exchange = 'binance/f'
instrument = 'LITUSDT'  #'BTCUSDT'#'ETHUSDT'


In [ ]:
def on_trade_fn(trade):
    print('>>> Got trade:\n', trade)


def on_order_fn(order):
    print('\n>>> Got order:\n', order.info())


def on_error_fn(error: Exception):
    # print(traceback.format_exc())
    print('>>> Got error: ', error)


In [ ]:
ah.get_stream(USER_EMAIL, SIGNKEY,
              exchange=exchange,
              instrument=instrument,
              on_trade=on_trade_fn,
              # on_order=on_order_fn,
              on_error=on_error_fn)
